In [16]:
import os, numpy as np, torch
from torch.utils.data import DataLoader, Subset
import torch.nn as nn
from torch.optim import lr_scheduler
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18, ResNet18_Weights
from torchvision import transforms
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import StratifiedShuffleSplit
from pathlib import Path
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import time, copy, math
from collections import Counter

In [ ]:
# STEP1: Set up directories
training_dir = Path("C:/Users/tomla/Documents/Projects/brain_tumor_classifier/data/training/")
testing_dir = Path("C:/Users/tomla/Documents/Projects/brain_tumor_classifier/data/testing/")
OUTPUT_MODELS = Path("C:/Users/tomla/Documents/Projects/brain_tumor_classifier/models/")
OUTPUT_MODELS.mkdir(parents=True, exist_ok=True)

## Data Preprocessing

In [8]:
# STEP 2: Set up transforms for image Normalization and Augmentation

# Image Normalization paramters for ResNet18
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD  = (0.229, 0.224, 0.225)

data_transforms = {
    'train': transforms.Compose([
        transforms.Grayscale(num_output_channels=3),
        transforms.RandomRotation(15), 
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.1, contrast=0.1),
        transforms.ToTensor(),
        transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)
    ]),
    'val': transforms.Compose([
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)
    ]),
    'test': transforms.Compose([
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)
    ]),
}

In [10]:
# STEP 3: Create Dataloaders

# Load full training set
full_train = ImageFolder(training_dir, transform=data_transforms["train"]) # load full training dataset and assigns label to each image based on folder class
y = np.array([label for _, label in full_train.samples]) # extract labels

# Create stratified split (80% train / 20% val)
ss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(ss.split(np.zeros(len(y)), y)) # create indices for train/val split (requires X, y so zero array is used as X)

# training set
train_data = Subset(full_train, train_idx)

# validation set
val_train = ImageFolder(training_dir, transform=data_transforms["val"]) # load full training dataset without agumentations
val_data = Subset(val_train, val_idx)

# final test set
test_data = ImageFolder(testing_dir, transform=data_transforms["test"])

# Loaders
BATCH_SIZE = 32
NUM_WORKERS = 4

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

class_names = full_train.classes  # ordered by subfolder name
num_classes = len(class_names)
print("Classes:", class_names)


Classes: ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']


In [14]:
# STEP 4: Load pretrained Model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

weights = ResNet18_Weights.IMAGENET1K_V1
model = resnet18(weights=weights)

# replace final layer for 4 classes
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes)

model = model.to(device)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\tomla/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:01<00:00, 46.5MB/s]


In [17]:
# STEP 5: Run Train/validate loop with metrics

criterion = nn.CrossEntropyLoss()

# Optimizer / scheduler
import torch.optim as optim
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4, weight_decay=1e-4)
scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)  # gentle decay

def evaluate(model, loader, device):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            logits = model(xb)
            preds = torch.argmax(logits, dim=1)
            all_preds.append(preds.cpu())
            all_labels.append(yb.cpu())
    y_true = torch.cat(all_labels).numpy()
    y_pred = torch.cat(all_preds).numpy()
    acc = accuracy_score(y_true, y_pred)
    prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="macro", zero_division=0)
    return acc, prec, rec, f1, y_true, y_pred

EPOCHS = 15
best_wts = copy.deepcopy(model.state_dict())
best_f1 = -math.inf

scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

for epoch in range(1, EPOCHS+1):
    t0 = time.time()
    # ---- train ----
    model.train()
    running_loss = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
            logits = model(xb)
            loss = criterion(logits, yb)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        running_loss += loss.item() * xb.size(0)

    scheduler.step()
    train_loss = running_loss / len(train_loader.dataset)

    # ---- validate ----
    val_acc, val_prec, val_rec, val_f1, y_true, y_pred = evaluate(model, val_loader, device)

    # track best
    if val_f1 > best_f1:
        best_f1 = val_f1
        best_wts = copy.deepcopy(model.state_dict())
        torch.save(best_wts, OUTPUT_MODELS / f"resnet18_best_valF1_{best_f1:.3f}.pth")

    dt = time.time() - t0
    print(f"[{epoch:02d}/{EPOCHS}] {dt:.1f}s  TrainLoss={train_loss:.4f}  "
          f"Val: Acc={val_acc:.3f}  P={val_prec:.3f}  R={val_rec:.3f}  F1={val_f1:.3f}")

# load best weights
model.load_state_dict(best_wts)


C:\Users\tomla\AppData\Local\Temp\ipykernel_54716\3327404457.py:30: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())
C:\Users\tomla\AppData\Local\Temp\ipykernel_54716\3327404457.py:40: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):


[01/15] 18.3s  TrainLoss=0.4651  Val: Acc=0.908  P=0.913  R=0.903  F1=0.908


C:\Users\tomla\AppData\Local\Temp\ipykernel_54716\3327404457.py:40: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):


[02/15] 17.6s  TrainLoss=0.1610  Val: Acc=0.936  P=0.940  R=0.931  F1=0.935


C:\Users\tomla\AppData\Local\Temp\ipykernel_54716\3327404457.py:40: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):


...

C:\Users\tomla\AppData\Local\Temp\ipykernel_54716\3327404457.py:40: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):


[14/15] 17.3s  TrainLoss=0.0075  Val: Acc=0.962  P=0.961  R=0.952  F1=0.956


C:\Users\tomla\AppData\Local\Temp\ipykernel_54716\3327404457.py:40: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):


[15/15] 17.6s  TrainLoss=0.0076  Val: Acc=0.965  P=0.966  R=0.958  F1=0.962


<All keys matched successfully>

In [18]:
# STEP 6: Final model evaluation on test set
test_acc, test_prec, test_rec, test_f1, y_true, y_pred = evaluate(model, test_loader, device)
print(f"TEST — Acc={test_acc:.3f}  P={test_prec:.3f}  R={test_rec:.3f}  F1={test_f1:.3f}")
print(classification_report(y_true, y_pred, target_names=class_names, zero_division=0))

TEST — Acc=0.774  P=0.839  R=0.761  F1=0.752
                  precision    recall  f1-score   support

    glioma_tumor       0.92      0.35      0.51       100
meningioma_tumor       0.69      0.99      0.81       115
        no_tumor       0.74      0.97      0.84       105
 pituitary_tumor       1.00      0.73      0.84        74

        accuracy                           0.77       394
       macro avg       0.84      0.76      0.75       394
    weighted avg       0.82      0.77      0.75       394



## Troubleshooting

In [ ]:
# STEP 1: Check index remap
print("TRAIN mapping:", full_train.class_to_idx)
print("TEST  mapping:", test_data.class_to_idx)

# Must match by class name → same index
assert set(full_train.classes) == set(test_data.classes)

# If order differs, build an index remap for evaluation
train_map = full_train.class_to_idx           # {'glioma':0, 'meningioma':1, ...}
test_map  = test_data.class_to_idx              # may be different order

idx_remap = {test_idx: train_map[name] for name, test_idx in test_map.items()}
print("Index remap:", idx_remap)

TRAIN mapping: {'glioma_tumor': 0, 'meningioma_tumor': 1, 'no_tumor': 2, 'pituitary_tumor': 3}
TEST  mapping: {'glioma_tumor': 0, 'meningioma_tumor': 1, 'no_tumor': 2, 'pituitary_tumor': 3}
Index remap: {0: 0, 1: 1, 2: 2, 3: 3}
